# U1 Actividad NLP

En esta actividad se ha analizado el conjunto de datos de [OpinRankDatasetWithJudgments](http://kavita-ganesan.com/entity-ranking-data/) para hallar las respuestas a las preguntas que propone el enunciado.

Primero, nos descargamos el dataset:

In [ ]:
# If you haven't downloaded the dataset, then uncomment and run. Note unzipping is os dependent
# !curl https://github.com/kavgan/OpinRank/raw/master/OpinRankDatasetWithJudgments.zip -o "data/OpinRankDatasetWithJudgments.zip"
# !7z x -y "data/OpinRankDatasetWithJudgments.zip" -odata # 7zip / windows
# !unzip "data/OpinRankDatasetWithJudgments.zip" # mac

In [ ]:
from ipyparallel import Client
rc = Client()

## Lectura de datos

Dentro del zip que hemos descargados encontramos la siguiente estructura de ficheros:

- data (dir)
  - cars (dir)
  - hotels (dir)
    - data (dir)
      - city-name (dir)
        - hotel-name (file)
          - *reviews here*
      - city-name.csv (file)
        - *hotels information*
    - judgments
      - city-name

Ignoramos los datos de coches, directorio `cars`, y nos centraremos en el directorio `hotels`. Dentro de `hotels` encontramos dos directorios, `data` y `judgments`. 

En el primer directorio, data, encontramos los datos sin procesar. Estos datos se agrupan por ciudades. Cada ciudad tiene un fichero `.csv` y un directorio con su nombre. En el csv encontramos la lista de hoteles de esa ciudad y datos asociados a estos. Entre estos datos destaca el atributo `doc_id`, que contiene el nombre del fichero donde se encuentran las review de los usuarios. El fichero de las review se encuentra dentro del directorio con el nombre de la ciudad a la que pertenece.

El `judgments` encontramos un directorio por cada ciudad. Dentro de cada directorio encontramos los resultados de las puntuaciones de relevancia.

Para información más detallada podéis consultar la documentación del dataset, `OpinRankDatasetWithJudgments.pdf`.

In [ ]:
import os
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

hotels = pd.DataFrame()

# Podemos iterar y unir todos los csv porque sabemos que solo hay 10 ciudades.
# Alternativamente, usariamos una función para cargar cada ciudad de forma independiente 
# como se ha hecho con las review de cada hotel.
for name in os.listdir("data/hotels/data"):
    if name.endswith(".csv"):
      hotelTmp = pd.read_csv(f"data/hotels/data/{name}", delimiter=',', index_col=False)
      hotels = pd.concat([hotels, hotelTmp], axis=0)

# Guardamos todos los nombres de las ciudades
cities = hotels.city.unique()

hotels.head()

Como hay muchas reviews vamos a cargar el listado de reviews de cada hotel a demanda.

In [ ]:
%%px
# from textblob import TextBlob
# def spellcheck(inputText):
#   blob = TextBlob(str(inputText))
#   return str(blob.correct())

# Dada una fila/row del dataframe hotels del apartado anterior, nos devuelve un dataframe con las reviews
def getReviews(hotel):
  docId = hotel["doc_id"]
  city = hotel["city"].replace(" ", "-")
  fileName = f"data/hotels/data/{city}/{docId}"
  try:
    reviews = pd.read_csv(fileName, delimiter="\t", index_col=False, header=None, encoding="cp1252", on_bad_lines='skip')
    reviews.columns = ["date", "title", "review", "empty"]

    # la columna de date no la vamos a usar y la ultima columna, empty, es resultado de tener un tabulador
    # a final de cada linea. No hay datos. Por lo tanto las quitamos
    reviews = reviews.drop(columns=["date", "empty"]) 

    # corregir ortografía. Es muuuuy lento
    # reviews["title"] = reviews["title"].apply(spellcheck)
    # reviews["review"] = reviews["review"].apply(spellcheck)

    return reviews
  except FileNotFoundError:
    return pd.DataFrame()
  except Exception as e:
    print(e)
    return pd.DataFrame()

In [ ]:
%%px --targets 0
# test the function
reviews = getReviews(hotels.iloc[0])
print(f"cantidad de reviews: {len(reviews)}")
reviews.head()

En el proceso de carga de las reviews vimos que los ficheros que contienen las review:
- Tienen una review por fila.
- El separador es un tabulador.
- El encode del fichero es `cp1252`. Se puede abrir en `utf-8` y es legible excepto acentos y algunos caracteres especiales.

En el ejemplo podemos comprobar que la columna `date` no siempre va a tener valor.

Actualización:
  - Se ha modificado el algoritmo para que incluya soporte para corregir typos y se ha quitado la columna `date` ya que no aporta valor.
  - Se ha comentado el corrector. Se tarda en mi maquina unos 10minutos para obtener 250 reviews. Sin el corrector son 0.6s.

## 1. ¿Qué partes de una habitación son las más mencionadas?

In this section, I'm going to use wordnet of each room to look for matches. First, I'm going to look for the definition of room that we need. Then use it's hyponyms to search in the reviews.

In [ ]:
%%px
# load dependencies
import re
import tqdm
# import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
# nltk.download("punkt")
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pprint

lemmatizer = WordNetLemmatizer()

In [ ]:
%%px --target 0
# check the room definition that we need
for sense in wordnet.synsets('room'):
    print(sense)
    print(sense.definition())
    print(sense.examples())
    print("-"*10)

# Synset('room.n.01') is the definition that I'm looking for

In [ ]:
%%px --target 0
room = wordnet.synset('room.n.01')
room.hyponyms()

Estas funciones nos servirán para lanzar y recuperar los procesos en paralelo

In [ ]:
import functools

# reduce function
def concatCounter(dict1, dict2):
  for key in dict1.keys():
    if key in dict2:
      dict2[key] += dict1[key]
    else:
      dict2[key] = dict1[key]
  return dict2

def parallelize_dataframe(df, func, reduceFunc):
  df_split = np.array_split(df, os.cpu_count())
  # get a view on the cluster
  view = rc.load_balanced_view()
  # submit the tasks
  asyncresult = view.map_async(func, df_split)
  # wait interactively for results
  asyncresult.wait_interactive()
  # retrieve actual results
  result = functools.reduce(reduceFunc,asyncresult.get())
  return result

Las funciones a continuación serán usadas en cada thread / cluster.
En el primer bloque hay las genéricas.
En el segundo bloque las funciones de búsqueda de habitaciones

In [ ]:
%%px 
def addOne(dict, key):
  if key in dict:
    dict[key] += 1
  else:
    dict[key] = 1
  return dict

def getNouns(review):
  nouns = []
  cleanText = wordCleaner(review)
  tokens = list(set(word_tokenize(cleanText)))
  taggedTokens = pos_tag(tokens, tagset='universal')
  for taggedToken in taggedTokens:
    (word, tokenType) = taggedToken
    if tokenType == "NOUN":
      nouns.append(word)
  return nouns

def wordCleaner(text):
  return re.sub("(\W(?!(\w)))+", " ", str(text)).lower()

def getSynsetsNames(synsets):
  return list(map(lambda synset: synset.name(), synsets))

rooms = getSynsetsNames(wordnet.synset('room.n.01').hyponyms()) # get room names as "constant", so we don't recalculate every time

In [ ]:
%%px
def processReview(counter, review):
  text = str(review.title) + " " + str(review.review)
  tokenizedText = getNouns(text)
  for word in tokenizedText:
    try:
      roomType = getRoomType(word)
      if roomType != None:
        counter = addOne(counter, roomType)
    except Exception as e:
      print(f"failed with word '{word}' with type '{roomType}' and error code: {e}")
  return counter

def getRoomType(word):
  wordSynsets = wordnet.synsets(str(word))
  for wordSynset in getSynsetsNames(wordSynsets):
    if wordSynset in rooms:
      return wordSynset
  return None

La función `findAllRoomMentions` al ser la principal la definimos en local porque debe ser visible para la función `map` (dentro de `parallelize_dataframe`) y enviarla al cluster a ejecutar.

In [ ]:
def findAllRoomMentions(hotels):
  counter = {}
  for index, hotel in hotels.iterrows():
    reviews = getReviews(hotel)
    for reviewIdx, review in reviews.iterrows():
      counter = processReview(counter, review)
  return counter

Usamos 12 hoteles para probar que los resultados son los que queremos.

In [ ]:
# test
parallelize_dataframe(hotels[0:12], findAllRoomMentions, concatCounter)

Ejecutamos el proceso para todos los hoteles agrupados por ciudad

In [ ]:
stats = {}

for city in cities:
  stats[city] =  parallelize_dataframe(hotels[hotels["city"] == city], findAllRoomMentions, concatCounter)

statsDf = pd.DataFrame(stats).replace('NaN', 0)
statsDf.to_csv('resultados_apartado_1.csv')
statsDf.plot.barh(figsize=(16,10), stacked=True)

### Resultados

## 2. ¿Qué servicios pueden detectarse por cada hotel?
Ejecutamos el mismo código de búsqueda anterior mortificándolo para que esta vez busque servicios.

Antes de que pueda buscar servicios debemos definir cuales son los *synset* de los servicios de un hotel.

In [ ]:
%%px --target 0
# buscar las definiciones de "que es un servicio"
hotelServices = ["restaurant", "pool", "spa", "gym", "bellman", "wifi", "television", "excursion", "clean", "hotspot"]
for service in hotelServices:
  for sense in wordnet.synsets(service):
      print(sense)
      print(sense.definition())
      print(sense.examples())
      print(sense.hyponyms())
      print(sense.hypernyms())
      print("-"*10)

Los servicios que hemos seleccionado son los siguientes: // TODO: explain why
- service.n.15
- restaurant.n.01
- pool.n.01
- watering_place.n.01 # kinda spa
- athletic_facility.n.01
- baggageman.n.01
- bellboy.n.01
- lifeguard.n.01
- checker.n.01
- houseclean.v.01
- wireless_local_area_network.n.01
- television.n.01
- excursion.n.01
- gambling_house.n.01 
- health_spa.n.01 
- hotel-casino.n.01 
- massage_parlor.n.02 
- mercantile_establishment.n.01
- spot.n.07

In [ ]:
%%px
services = [
  wordnet.synset('service.n.15'), # el servicio que da un camarero a los clientes
  wordnet.synset('restaurant.n.01'),
  wordnet.synset('pool.n.01'),
  wordnet.synset('watering_place.n.01'), # kinda spa
  wordnet.synset('athletic_facility.n.01'),
  wordnet.synset('baggageman.n.01'),
  wordnet.synset('bellboy.n.01'),
  wordnet.synset('lifeguard.n.01'),
  wordnet.synset('checker.n.01'),
  wordnet.synset('houseclean.v.01'),
  wordnet.synset('wireless_local_area_network.n.01'),
  wordnet.synset('television.n.01'),
  wordnet.synset('excursion.n.01'),
  wordnet.synset('gambling_house.n.01'), 
  wordnet.synset('health_spa.n.01'), 
  wordnet.synset('hotel-casino.n.01'), 
  wordnet.synset('massage_parlor.n.02'), 
  wordnet.synset('mercantile_establishment.n.01'),
  wordnet.synset('spot.n.07'),
]
serviceNames = getSynsetsNames(services)

Creamos la función `getAllHypernyms` capaz de calcular todos los *hypernyms* dada una palabra

In [ ]:
%%px
from collections import OrderedDict

def removeDuplicates(list):
  return OrderedDict.fromkeys(list).keys()

def getAllHypernyms(itemName):
  allItemHypernyms = []
  for synset in wordnet.synsets(itemName):
    allItemHypernyms = allItemHypernyms + getAllHypernymsOsSynset(synset)
  return removeDuplicates(allItemHypernyms)

def getAllHypernymsOsSynset(synset):
  synsetHypernyms = synset.hypernyms()
  if len(synsetHypernyms) == 0:
    return []

  allSynsetHypernyms = []
  for hypernym in synsetHypernyms:
    allSynsetHypernyms = allSynsetHypernyms + getAllHypernymsOsSynset(hypernym)

  return synsetHypernyms + allSynsetHypernyms

In [ ]:
%%px --target 0
# test
pprint.pprint(getAllHypernyms('restaurant'))

Procedimiento de búsqueda

In [ ]:
%%px
def lookForServices(counter, review):
  text = str(review.title) + " " + str(review.review)
  tokenizedText = getNouns(text)
  for word in tokenizedText:
    try:
      serviceType = getServiceType(word)
      if serviceType != None:
        counter = addOne(counter, word)
    except Exception as e:
      print(e)
  return counter

# En este caso en lugar de mirar si los synset de 1 nivel encajan con la lista de habitaciones posibles, vamos a hacerlo al revés.
# Vamos a coger todos los hypernyms de la palabra y ver si encaja 
def getServiceType(word):
  wordSynsets = getAllHypernyms(word)
  for wordSynset in getSynsetsNames(wordSynsets):
    if wordSynset in serviceNames:
      return wordSynset
  return None

In [ ]:
def findAllServices(hotels):
  counterList = {}
  for index, hotel in tqdm.tqdm(hotels.iterrows()):
    reviews = getReviews(hotel)
    counter = {}
    for reviewIdx, review in reviews.iterrows():
      lookForServices(counter, review)
    counterList[hotel["hotel_name"]] = counter
  return counterList

# test
result = parallelize_dataframe(hotels[0:12], findAllServices, concatCounter)
pd.DataFrame(result).head()

In [ ]:
result = parallelize_dataframe(hotels, findAllServices, concatCounter)
result = {k: v for k, v in result.items() if v} # remove empty results
result2 = pd.DataFrame(result).transpose()
result2.to_csv('resultados_apartado_2.csv')
result2.head()

### Resultados

Los servicios encontrados en las review de cada hotel no tiene porque ser del propio hotel.

Comentar también que el synset de 'pool.n.01' puesto para detectar si el hotel tiene o no piscina, ha terminado detectando si hay desagües (sink/s)

## 3. ¿Qué lugares y qué porcentaje de revisiones sobre cada ciudad mencionan otras ciudades, regiones o puntos de interés turístico?

En este apartado en lugar de usar nltk vamos a usar spacy. Spacy, durante la tokenización nos etiqueta el tipo de palabra que es. En nuestro caso buscaremos por las palabras que se etiqueten como `GPE`, "Geopolitical entity, i.e. countries, cities, states".

In [ ]:
%%px
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def findGpeMentions(hotels):
  gpeMentions = {}
  for index, hotel in hotels.iterrows():
    reviews = getReviews(hotel)
    for reviewIdx, review in reviews.iterrows():
      text = str(review.title) + ". " + str(review.review)
      cleanText = wordCleaner(text)
      tokens = nlp(cleanText)
      for ent in tokens.ents:
        if ent.label_ == "GPE":
          gpeMentions = addOne(gpeMentions, ent.lower_)
      gpeMentions = addOne(gpeMentions, "reviewsAnalyzed")
  return gpeMentions

#test
test_result = parallelize_dataframe(hotels[10:12], findGpeMentions, concatCounter)
pprint.pprint(test_result)

In [ ]:
allGpeMentions = {}

for city in cities:
  allGpeMentions[city] = parallelize_dataframe(hotels[hotels["city"] == city], findGpeMentions, concatCounter)

allGpeMentionsDf = pd.DataFrame.from_dict(allGpeMentions).fillna(0)
allGpeMentionsDf.to_csv("resultados_apartado_3.csv")
allGpeMentionsDf.head()

### Por ejemplo, con vuestra información podríamos llegar a responder a la pregunta: ¿en Las Vegas se menciona el Gran Cañón del Colorado?
Si, para ello tendríamos que buscar en el resultado de búsqueda `allGpeMentions` por la key, `Las Vegas` y dentro de las menciones de Las Vegas buscar si existe la key `Gran Cañón del Colorado`, en el idoma de las reviews, inglés.
Hay que tener en cuenta que no se corregido el input del usuario, por lo que resultados como "Gran canyon" o "Colorado canyon" aparecerían como sitios distintos.

In [ ]:
lasVegasGpe = allGpeMentions.get("las vegas", {})
granCanyonGpe = lasVegasGpe.get("gran canyon", "No se ha encontrado 'gran canyon'")
pprint.pprint(lasVegasGpe)

Buscando con regex a mano si he encontrado "gran canyon". A continuación el porqué no se detecta.

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

# review 320 de usa_nevada_las-vegas_excalibur_hotel_casino
text = "fantastic value for money at the excalibur	My husband and i stayed at the excalibur from the 6th to 16th april we had a wonderful time, this hotel is in a great spot on strip, we were in tower 2 and the room was spotless, cleaned every day, food was great value from buffet bar, staff were very friendly and helpful, if you are going to visit gran canyon book with sundance they are a class act just a fantastic place to visit las vegas cant wait to go back"

train = nlp(text)
displacy.render(train, jupyter=True, style="ent")

Gran canyon se etiqueta como nombre de persona en lugar de una ubicación, `GPE`.